In [2]:
#@title 授权挂载Google云盘
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
#@title 克隆（下载）存储库, install libnetcdf
%cd /content/gdrive/MyDrive
!git clone --recursive https://github.com/NOAA-GFDL/MOM6-examples.git MOM6-example
!sudo apt-get install libnetcdff-dev

/content/gdrive/MyDrive
Cloning into 'MOM6-example'...
remote: Enumerating objects: 67207, done.
remote: Counting objects: 100% (67207/67207), done.
remote: Compressing objects: 100% (8207/8207), done.
remote: Total 67207 (delta 58191), reused 66854 (delta 58081), pack-reused 0
Receiving objects: 100% (67207/67207), 66.06 MiB | 14.40 MiB/s, done.
Resolving deltas: 100% (58191/58191), done.
Updating files: 100% (3273/3273), done.
Submodule 'src/FMS1' (https://github.com/NOAA-GFDL/FMS) registered for path 'src/FMS1'
Submodule 'src/FMS2' (https://github.com/NOAA-GFDL/FMS) registered for path 'src/FMS2'
Submodule 'src/MOM6' (https://github.com/NOAA-GFDL/MOM6.git) registered for path 'src/MOM6'
Submodule 'src/SIS2' (https://github.com/NOAA-GFDL/SIS2.git) registered for path 'src/SIS2'
Submodule 'src/atmos_null' (https://github.com/NOAA-GFDL/atmos_null.git) registered for path 'src/atmos_null'
Submodule 'src/coupler' (https://github.com/NOAA-GFDL/coupler.git) registered for path 'src/coupler

In [16]:
#@title Prepare files
#MOM6 依赖于作为灵活运行时环境 （FRE） 一部分的工具来帮助完成构建过程。这些工具在 NOAA-GFDL/mkmf 存储库中提供，您应该在 中找到。MOM6-examples/src/mkmf/
# 我们在这些 wiki 页面中描述的工作流通常在用户创建的目录中编译模型：MOM6-
%cd /content/gdrive/MyDrive/MOM6-example
!mkdir build

!echo > build/env

!mkdir -p build/fms/
%cd build/fms/
!rm -f path_names
!sudo chmod -R 777 ../../src/mkmf/bin/list_paths
!sudo chmod -R 777 ../../src/mkmf/bin/mkmf
!../../src/mkmf/bin/list_paths -l ../../src/FMS
!../../src/mkmf/bin/mkmf -t ../../src/mkmf/templates/linux-ubuntu-jammy-gnu.mk -p libfms.a -c "-Duse_libMPI -Duse_netCDF -DSPMD" path_names

/content/gdrive/MyDrive/MOM6-example
mkdir: cannot create directory ‘build’: File exists
/content/gdrive/MyDrive/MOM6-example/build/fms
A list of the files you checked out is in the file path_names.
........................................................................................ Makefile is ready.


In [17]:
#@title Compiling the FMS shared code
%cd /content/gdrive/MyDrive/MOM6-example/build/fms/
!source ../env && make NETCDF=3 REPRO=1 libfms.a -j

!mkdir -p ../ocean_only/
%cd ../ocean_only/
!rm -f path_names
!../../src/mkmf/bin/list_paths -l ./ ../../src/MOM6/{config_src/infra/FMS1,config_src/memory/dynamic_symmetric,config_src/drivers/solo_driver,config_src/external,src/{*,*/*}}/
!../../src/mkmf/bin/mkmf -t ../../src/mkmf/templates/linux-ubuntu-jammy-gnu.mk -o '-I../fms' -p 'MOM6 -L../fms -lfms' -c "-Duse_libMPI -Duse_netCDF -DSPMD" path_names

/content/gdrive/MyDrive/MOM6-example/build/fms
mpif90 -Duse_libMPI -Duse_netCDF -DSPMD -Duse_netCDF -DHAVE_SCHED_GETAFFINITY -DHAVE_GETTID -Duse_LARGEFILE -I/usr/include -I/usr/include -fcray-pointer -fdefault-double-8 -fdefault-real-8 -Waliasing -ffree-line-length-none -fno-range-check -fallow-invalid-boz -fallow-argument-mismatch -O2 -fbounds-check   -c -I../../src/FMS/include	../../src/FMS/mpp/mpp_parameter.F90
mpif90 -Duse_libMPI -Duse_netCDF -DSPMD -Duse_netCDF -DHAVE_SCHED_GETAFFINITY -DHAVE_GETTID -Duse_LARGEFILE -I/usr/include -I/usr/include -fcray-pointer -fdefault-double-8 -fdefault-real-8 -Waliasing -ffree-line-length-none -fno-range-check -fallow-invalid-boz -fallow-argument-mismatch -O2 -fbounds-check   -c -I../../src/FMS/include	../../src/FMS/platform/platform.F90
mpicc -Duse_libMPI -Duse_netCDF -DSPMD -Duse_netCDF -DHAVE_SCHED_GETAFFINITY -DHAVE_GETTID -Duse_LARGEFILE  -I/usr/include -I/usr/include/hdf5/serial -D__IFC -O2   -c -I../../src/FMS/mosaic	../../src/FMS/mosaic/

In [18]:
#@title Compiling the mom6 Model
!mkdir -p /content/gdrive/MyDrive/MOM6-example/build/ocean_only/
%cd /content/gdrive/MyDrive/MOM6-example/build/ocean_only/
!rm -f path_names
!../../src/mkmf/bin/list_paths -l ./ ../../src/MOM6/{config_src/infra/FMS1,config_src/memory/dynamic_symmetric,config_src/drivers/solo_driver,config_src/external,src/{*,*/*}}/
!../../src/mkmf/bin/mkmf -t ../../src/mkmf/templates/linux-ubuntu-jammy-gnu.mk -o '-I../fms' -p 'MOM6 -L../fms -lfms' -c "-Duse_libMPI -Duse_netCDF -DSPMD" path_names
!source ../env && make NETCDF=3 REPRO=1 MOM6 -j

/content/gdrive/MyDrive/MOM6-example/build/ocean_only
A list of the files you checked out is in the file path_names.
................................................................................................................................................................................................................................................................................................ Makefile is ready.
mpif90 -Duse_libMPI -Duse_netCDF -DSPMD -Duse_netCDF -DHAVE_SCHED_GETAFFINITY -DHAVE_GETTID -Duse_LARGEFILE -I/usr/include -I/usr/include -fcray-pointer -fdefault-double-8 -fdefault-real-8 -Waliasing -ffree-line-length-none -fno-range-check -fallow-invalid-boz -fallow-argument-mismatch -O2 -fbounds-check -I../fms  -c	../../src/MOM6/src/parameterizations/CVmix/cvmix_kinds_and_types.F90
mpif90 -Duse_libMPI -Duse_netCDF -DSPMD -Duse_netCDF -DHAVE_SCHED_GETAFFINITY -DHAVE_GETTID -Duse_LARGEFILE -I/usr/include -I/usr/include -fcray-pointer -fdefault-double-8 -fdefault-real-

In [19]:
#@title Run the double_gyre mode
%cd /content/gdrive/MyDrive/MOM6-example/ocean_only/double_gyre

!mkdir -p RESTART
!OMPI_ALLOW_RUN_AS_ROOT=1 OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 mpirun -np 1 ../../build/ocean_only/MOM6

/content/gdrive/MyDrive/MOM6-example/ocean_only/double_gyre
NOTE: MPP_DOMAINS_SET_STACK_SIZE: stack size set to    32768.
&MPP_IO_NML
 HEADER_BUFFER_VAL=16384      ,
 GLOBAL_FIELD_ON_ROOT_PE=T,
 IO_CLOCKS_ON=F,
 SHUFFLE=0          ,
 DEFLATE_LEVEL=-1         ,
 CF_COMPLIANCE=F,
 /
NOTE: MPP_IO_SET_STACK_SIZE: stack size set to     131072.
NOTE: ======== Model being driven by MOM_driver ========
NOTE: callTree: o Program MOM_main, MOM_driver.F90
NOTE: open_param_file: MOM_input has been opened successfully.
NOTE: open_param_file: MOM_override has been opened successfully.
 MOM_in domain decomposition
whalo =    4, ehalo =    4, shalo =    4, nhalo =    4
  X-AXIS =   44
  Y-AXIS =   40
 MOM_inc domain decomposition
whalo =    2, ehalo =    2, shalo =    2, nhalo =    2
  X-AXIS =   22
  Y-AXIS =   20
NOTE: diag_manager_mod::diag_manager_init: prepend_date only supported when diag_manager_init is called with time_init present.
 MOM_in domain decomposition
whalo =    4, ehalo =    4, shal

In [8]:
!ls -lh /content/
!du -sh /content/MOM6-example

total 8.0K
drwx------ 5 root root 4.0K Dec 11 14:17 gdrive
drwxr-xr-x 1 root root 4.0K Dec  4 14:27 sample_data
du: cannot access '/content/MOM6-example': No such file or directory


In [9]:
!ls -lh /content/MOM6-example/ocean_only/double_gyre/RESTART

ls: cannot access '/content/MOM6-example/ocean_only/double_gyre/RESTART': No such file or directory


In [10]:
# 查看输入数据
!find /content/MOM6-example -name "MOM_input*"


find: ‘/content/MOM6-example’: No such file or directory


In [11]:
#查看nc文件信息
!ncdump -h /content/MOM6-example/ocean_only/double_gyre/RESTART/MOM.res.nc


/bin/bash: line 1: ncdump: command not found


In [12]:
#将结果输出为nc文件
from google.colab import files

files.download("/content/MOM6-example/ocean_only/double_gyre/RESTART/MOM.res.nc")


FileNotFoundError: ignored